# Census API Test & US Map Visualization

This notebook tests:
1. **Census ACS API** - Pull tract-level demographic data (population, median income, etc.)
2. **Census Geocoder** - Convert coordinates to FIPS tracts
3. **Visualize** the results on an interactive US choropleth map using Plotly

---
## 1. Setup & Load API Keys

In [ ]:
import os
import json
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load .env from project root
_cwd = os.getcwd()
_project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) or os.path.isfile(os.path.join(_cwd, ".env")) else os.path.dirname(_cwd)
_env_path = os.path.join(_project_root, ".env")

try:
    from dotenv import load_dotenv
    load_dotenv(_env_path)
    print(f".env loaded from: {_env_path}")
except ImportError:
    print("python-dotenv not installed; using environment variables")

CENSUS_API_KEY = os.environ.get("CENSUS_API_KEY", "")
print(f"Census API key: {'SET (' + CENSUS_API_KEY[:8] + '...)' if CENSUS_API_KEY else 'NOT SET'}")

---
## 2. Test Census ACS API -- State-Level Demographics

Pull population, median income, median home value, and median rent for **all US states** from the ACS 5-year dataset.

In [ ]:
# ---- Census ACS 5-Year: State-level data for all US states ----
year = "2022"
dataset = "acs5"
base_url = f"https://api.census.gov/data/{year}/acs/{dataset}"

# Variables:
#   B01003_001E = Total population
#   B19013_001E = Median household income
#   B25077_001E = Median home value
#   B25064_001E = Median gross rent
variables = "B01003_001E,B19013_001E,B25077_001E,B25064_001E,NAME"

params = {
    "get": variables,
    "for": "state:*",
}
if CENSUS_API_KEY:
    params["key"] = CENSUS_API_KEY

print(f"Requesting: {base_url}")
print(f"Params: for=state:*, variables={variables.split(',')[:-1]}")

r = requests.get(base_url, params=params, timeout=30)
print(f"\nStatus: {r.status_code}")
r.raise_for_status()

data = r.json()
cols = data[0]
rows = data[1:]

df_states = pd.DataFrame(rows, columns=cols)

# Convert numeric columns
rename_map = {
    "B01003_001E": "population",
    "B19013_001E": "median_income",
    "B25077_001E": "median_home_value",
    "B25064_001E": "median_rent",
}
df_states.rename(columns=rename_map, inplace=True)
for col in rename_map.values():
    df_states[col] = pd.to_numeric(df_states[col], errors="coerce")

print(f"\n--- Census ACS {year} -- State-Level Data ({len(df_states)} states/territories) ---")
print(f"SUCCESS! Retrieved data for {len(df_states)} states/territories.")
df_states.head(10)

---
## 3. Test Census ACS API -- Tract-Level Data (Sample: New York)

Pull tract-level demographics for New York state to verify tract-level access works.

In [ ]:
# ---- Census ACS: Tract-level for New York (state FIPS = 36) ----
state_fips = "36"  # New York
params_tract = {
    "get": variables,
    "for": "tract:*",
    "in": f"state:{state_fips}",
}
if CENSUS_API_KEY:
    params_tract["key"] = CENSUS_API_KEY

print(f"Fetching tract-level data for NY (state={state_fips})...")
r_tract = requests.get(base_url, params=params_tract, timeout=60)
print(f"Status: {r_tract.status_code}")
r_tract.raise_for_status()

data_tract = r_tract.json()
df_tracts_ny = pd.DataFrame(data_tract[1:], columns=data_tract[0])
df_tracts_ny.rename(columns=rename_map, inplace=True)
for col in rename_map.values():
    df_tracts_ny[col] = pd.to_numeric(df_tracts_ny[col], errors="coerce")

# Build full FIPS code
df_tracts_ny["tract_fips"] = df_tracts_ny["state"] + df_tracts_ny["county"] + df_tracts_ny["tract"]

print(f"\n--- Census ACS {year} -- NY Tracts ({len(df_tracts_ny)} tracts) ---")
print(f"SUCCESS! Retrieved {len(df_tracts_ny)} tracts for New York.")
print(f"\nSample stats:")
print(f"  Population range: {df_tracts_ny['population'].min():,.0f} -- {df_tracts_ny['population'].max():,.0f}")
print(f"  Median income range: ${df_tracts_ny['median_income'].min():,.0f} -- ${df_tracts_ny['median_income'].max():,.0f}")
df_tracts_ny.head(10)

---
## 4. Test Census Geocoder -- Coordinates to FIPS Tract

The Census Geocoder converts lat/lon coordinates into FIPS codes (state, county, tract). No API key required.

**Endpoint:** `https://geocoding.geo.census.gov/geocoder/geographies/coordinates`  
**Benchmark:** `Public_AR_Current`  
**Vintage:** `Census2020_Current`

In [ ]:
# ---- Census Geocoder: Test multiple coordinate pairs ----
test_locations = [
    {"name": "NYC - Times Square",       "lat": 40.7580, "lon": -73.9855},
    {"name": "Los Angeles - Downtown",    "lat": 34.0522, "lon": -118.2437},
    {"name": "Chicago - Loop",            "lat": 41.8781, "lon": -87.6298},
    {"name": "Houston - Downtown",        "lat": 29.7604, "lon": -95.3698},
    {"name": "Phoenix - Downtown",        "lat": 33.4484, "lon": -112.0740},
    {"name": "Philadelphia - City Hall",  "lat": 39.9526, "lon": -75.1652},
    {"name": "San Antonio - Alamo",       "lat": 29.4241, "lon": -98.4936},
    {"name": "Dallas - Downtown",         "lat": 32.7767, "lon": -96.7970},
]

# NOTE: endpoint must include /coordinates for lat/lon lookups
geocoder_url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
geocode_results = []

print("Testing Census Geocoder (coordinates -> FIPS tract)...")
print(f"Endpoint: {geocoder_url}\n")

for loc in test_locations:
    params_geo = {
        "x": loc["lon"],
        "y": loc["lat"],
        "benchmark": "Public_AR_Current",
        "vintage": "Census2020_Current",
        "format": "json",
    }
    try:
        r_geo = requests.get(geocoder_url, params=params_geo, timeout=30)
        r_geo.raise_for_status()
        geo_data = r_geo.json()
        
        # Extract geographies from response
        geographies = geo_data.get("result", {}).get("geographies", {})
        
        # Try multiple geography layer keys
        tract_info = {}
        for key in ["Census Tracts", "2020 Census Blocks", "2020 Census Tracts"]:
            layer = geographies.get(key, [])
            if layer:
                tract_info = layer[0]
                break
        
        state_fips_geo = tract_info.get("STATE", "")
        county_fips_geo = tract_info.get("COUNTY", "")
        tract_code = tract_info.get("TRACT", "")
        geoid = tract_info.get("GEOID", "")
        name = tract_info.get("NAME", tract_info.get("BASENAME", ""))
        
        result = {
            "location": loc["name"],
            "lat": loc["lat"],
            "lon": loc["lon"],
            "state_fips": state_fips_geo,
            "county_fips": county_fips_geo,
            "tract": tract_code,
            "geoid": geoid,
            "census_name": name,
            "status": "OK" if state_fips_geo else "NO MATCH",
        }
        geocode_results.append(result)
        status_icon = "OK" if state_fips_geo else "FAIL"
        print(f"  [{status_icon}] {loc['name']:30s} -> State: {state_fips_geo}, County: {county_fips_geo}, Tract: {tract_code}, GEOID: {geoid}")
    except Exception as e:
        print(f"  [ERR] {loc['name']:30s} -> {e}")
        geocode_results.append({"location": loc["name"], "lat": loc["lat"], "lon": loc["lon"],
                                "state_fips": "", "county_fips": "", "tract": "", "geoid": "",
                                "census_name": "", "status": f"ERROR: {e}"})

df_geocoded = pd.DataFrame(geocode_results)
print(f"\n--- Geocoder Results ({len(df_geocoded)} locations) ---")
ok_count = (df_geocoded["status"] == "OK").sum()
print(f"Successful: {ok_count}/{len(df_geocoded)}")
df_geocoded

---
## 5. Visualize -- US Choropleth Map: Median Income by State

Interactive Plotly map showing Census ACS median household income across all US states.

In [ ]:
# ---- State FIPS -> State abbreviation mapping ----
fips_to_abbr = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA",
    "08": "CO", "09": "CT", "10": "DE", "11": "DC", "12": "FL",
    "13": "GA", "15": "HI", "16": "ID", "17": "IL", "18": "IN",
    "19": "IA", "20": "KS", "21": "KY", "22": "LA", "23": "ME",
    "24": "MD", "25": "MA", "26": "MI", "27": "MN", "28": "MS",
    "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND",
    "39": "OH", "40": "OK", "41": "OR", "42": "PA", "44": "RI",
    "45": "SC", "46": "SD", "47": "TN", "48": "TX", "49": "UT",
    "50": "VT", "51": "VA", "53": "WA", "54": "WV", "55": "WI",
    "56": "WY", "72": "PR",
}

df_states["state_abbr"] = df_states["state"].map(fips_to_abbr)
df_map = df_states.dropna(subset=["state_abbr", "median_income"])

print(f"Mapping {len(df_map)} states with valid income data...")

In [ ]:
# ---- Map 1: Median Household Income by State ----
fig_income = px.choropleth(
    df_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="median_income",
    hover_name="NAME",
    hover_data={
        "median_income": ":$,.0f",
        "population": ":,.0f",
        "median_home_value": ":$,.0f",
        "median_rent": ":$,.0f",
        "state_abbr": False,
    },
    color_continuous_scale="Viridis",
    scope="usa",
    title=f"Census ACS {year} -- Median Household Income by State",
    labels={"median_income": "Median Income ($)"},
)
fig_income.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=900,
    height=550,
)
fig_income.show()

In [ ]:
# ---- Map 2: Population by State ----
fig_pop = px.choropleth(
    df_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="population",
    hover_name="NAME",
    hover_data={
        "population": ":,.0f",
        "median_income": ":$,.0f",
        "state_abbr": False,
    },
    color_continuous_scale="YlOrRd",
    scope="usa",
    title=f"Census ACS {year} -- Total Population by State",
    labels={"population": "Population"},
)
fig_pop.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=900,
    height=550,
)
fig_pop.show()

In [ ]:
# ---- Map 3: Median Home Value by State ----
fig_home = px.choropleth(
    df_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="median_home_value",
    hover_name="NAME",
    hover_data={
        "median_home_value": ":$,.0f",
        "median_income": ":$,.0f",
        "median_rent": ":$,.0f",
        "state_abbr": False,
    },
    color_continuous_scale="Blues",
    scope="usa",
    title=f"Census ACS {year} -- Median Home Value by State",
    labels={"median_home_value": "Home Value ($)"},
)
fig_home.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=900,
    height=550,
)
fig_home.show()

---
## 6. Deep Dive -- County-Level Demographics (All US Counties)

Pull **all ~3,200 US counties** from Census ACS and visualize on an interactive choropleth map using county FIPS codes.

In [ ]:
# ---- Census ACS: County-level for ALL US states ----
import time

county_base_url = f"https://api.census.gov/data/{year}/acs/{dataset}"
county_variables = "B01003_001E,B19013_001E,B25077_001E,B25064_001E,NAME"

all_county_rows = []
county_cols = None

# Fetch county data state-by-state (API requires state filter for county)
state_fips_list = [f"{i:02d}" for i in range(1, 57)]
# Add PR (72)
state_fips_list.append("72")
# Remove non-existent FIPS (03, 07, 14, 43, 52)
skip_fips = {"03", "07", "14", "43", "52"}
state_fips_list = [s for s in state_fips_list if s not in skip_fips]

print(f"Fetching county-level data for {len(state_fips_list)} states/territories...")
errors = []
for idx, st in enumerate(state_fips_list):
    params_county = {
        "get": county_variables,
        "for": "county:*",
        "in": f"state:{st}",
    }
    if CENSUS_API_KEY:
        params_county["key"] = CENSUS_API_KEY
    try:
        r_county = requests.get(county_base_url, params=params_county, timeout=30)
        r_county.raise_for_status()
        data_county = r_county.json()
        if len(data_county) > 1:
            if county_cols is None:
                county_cols = data_county[0]
            all_county_rows.extend(data_county[1:])
    except Exception as e:
        errors.append(f"State {st}: {e}")
    # Small delay to be respectful to the API
    if idx % 10 == 9:
        time.sleep(0.5)

print(f"Fetched {len(all_county_rows)} counties total")
if errors:
    print(f"Errors ({len(errors)}): {errors[:5]}")

# Build DataFrame
df_counties = pd.DataFrame(all_county_rows, columns=county_cols)
df_counties.rename(columns=rename_map, inplace=True)
for col in rename_map.values():
    df_counties[col] = pd.to_numeric(df_counties[col], errors="coerce")

# Build 5-digit county FIPS (state + county)
df_counties["county_fips"] = df_counties["state"] + df_counties["county"]

print(f"\n--- Census ACS {year} -- County-Level ({len(df_counties)} counties) ---")
print(f"SUCCESS! Retrieved {len(df_counties)} counties across the US.")
print(f"\nCounty-level summary:")
print(df_counties[["population", "median_income", "median_home_value", "median_rent"]].describe().round(0))
df_counties.head(10)

In [ ]:
# ---- Map 5: County-Level Median Income Choropleth ----
# Plotly uses GeoJSON for county-level maps. We'll use the built-in US county FIPS support.
from urllib.request import urlopen

# Load US counties GeoJSON (Plotly's recommended source)
print("Loading US counties GeoJSON...")
with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties_geojson = json.loads(response.read().decode())
print(f"Loaded GeoJSON with {len(counties_geojson['features'])} county features")

# Filter out rows with missing income
df_county_map = df_counties.dropna(subset=["median_income"]).copy()
# Filter out negative sentinel values (Census uses -666666666 for missing)
df_county_map = df_county_map[df_county_map["median_income"] > 0]

fig_county_income = px.choropleth(
    df_county_map,
    geojson=counties_geojson,
    locations="county_fips",
    color="median_income",
    hover_name="NAME",
    hover_data={
        "median_income": ":$,.0f",
        "population": ":,.0f",
        "median_home_value": ":$,.0f",
        "median_rent": ":$,.0f",
        "county_fips": True,
    },
    color_continuous_scale="Viridis",
    scope="usa",
    title=f"Census ACS {year} -- Median Household Income by County",
    labels={"median_income": "Median Income ($)", "county_fips": "County FIPS"},
)
fig_county_income.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000,
    height=600,
)
fig_county_income.show()
print(f"Mapped {len(df_county_map)} counties with valid income data")

In [ ]:
# ---- Map 6: County-Level Population Choropleth ----
df_county_pop = df_counties.dropna(subset=["population"]).copy()
df_county_pop = df_county_pop[df_county_pop["population"] > 0]

fig_county_pop = px.choropleth(
    df_county_pop,
    geojson=counties_geojson,
    locations="county_fips",
    color="population",
    hover_name="NAME",
    hover_data={
        "population": ":,.0f",
        "median_income": ":$,.0f",
        "county_fips": True,
    },
    color_continuous_scale="YlOrRd",
    range_color=[0, df_county_pop["population"].quantile(0.95)],  # Cap at 95th percentile for better color spread
    scope="usa",
    title=f"Census ACS {year} -- Population by County (color capped at 95th percentile)",
    labels={"population": "Population", "county_fips": "County FIPS"},
)
fig_county_pop.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000,
    height=600,
)
fig_county_pop.show()

In [ ]:
# ---- Map 7: County-Level Median Home Value ----
df_county_hv = df_counties.dropna(subset=["median_home_value"]).copy()
df_county_hv = df_county_hv[df_county_hv["median_home_value"] > 0]

fig_county_hv = px.choropleth(
    df_county_hv,
    geojson=counties_geojson,
    locations="county_fips",
    color="median_home_value",
    hover_name="NAME",
    hover_data={
        "median_home_value": ":$,.0f",
        "median_income": ":$,.0f",
        "median_rent": ":$,.0f",
        "county_fips": True,
    },
    color_continuous_scale="Blues",
    range_color=[0, df_county_hv["median_home_value"].quantile(0.95)],
    scope="usa",
    title=f"Census ACS {year} -- Median Home Value by County",
    labels={"median_home_value": "Home Value ($)", "county_fips": "County FIPS"},
)
fig_county_hv.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000,
    height=600,
)
fig_county_hv.show()

In [ ]:
# ---- County Top/Bottom Analysis ----
print("=" * 80)
print(f"COUNTY-LEVEL DEEP DIVE -- Census ACS {year}")
print("=" * 80)

print(f"\nTotal counties retrieved: {len(df_counties)}")
df_valid = df_counties[df_counties["median_income"] > 0].copy()

print(f"\n--- Top 20 Counties by Median Income ---")
top20_county = df_valid.nlargest(20, "median_income")[["NAME", "county_fips", "median_income", "population", "median_home_value"]].copy()
top20_county["median_income"] = top20_county["median_income"].apply(lambda x: f"${x:,.0f}")
top20_county["population"] = top20_county["population"].apply(lambda x: f"{x:,.0f}")
top20_county["median_home_value"] = top20_county["median_home_value"].apply(lambda x: f"${x:,.0f}" if pd.notna(x) and x > 0 else "N/A")
print(top20_county.to_string(index=False))

print(f"\n--- Bottom 20 Counties by Median Income (pop > 10,000) ---")
df_pop_filter = df_valid[df_valid["population"] > 10000]
bot20_county = df_pop_filter.nsmallest(20, "median_income")[["NAME", "county_fips", "median_income", "population", "median_home_value"]].copy()
bot20_county["median_income"] = bot20_county["median_income"].apply(lambda x: f"${x:,.0f}")
bot20_county["population"] = bot20_county["population"].apply(lambda x: f"{x:,.0f}")
bot20_county["median_home_value"] = bot20_county["median_home_value"].apply(lambda x: f"${x:,.0f}" if pd.notna(x) and x > 0 else "N/A")
print(bot20_county.to_string(index=False))

print(f"\n--- Most Populous 20 Counties ---")
top20_pop = df_valid.nlargest(20, "population")[["NAME", "county_fips", "population", "median_income", "median_home_value"]].copy()
top20_pop["population"] = top20_pop["population"].apply(lambda x: f"{x:,.0f}")
top20_pop["median_income"] = top20_pop["median_income"].apply(lambda x: f"${x:,.0f}")
top20_pop["median_home_value"] = top20_pop["median_home_value"].apply(lambda x: f"${x:,.0f}" if pd.notna(x) and x > 0 else "N/A")
print(top20_pop.to_string(index=False))

---
## 7. Deep Dive -- ZIP Code (ZCTA) Level Demographics

Census doesn't use ZIP codes directly -- it uses **ZCTAs (ZIP Code Tabulation Areas)** which approximate USPS ZIP codes. We pull all ~33,000 ZCTAs nationwide and visualize them.

**Note:** ZCTA-level data covers the entire US and is very granular. The API returns all ZCTAs in one call (no state filter needed).

In [ ]:
# ---- Census ACS: ZCTA (ZIP Code Tabulation Area) level for entire US ----
zcta_base_url = f"https://api.census.gov/data/{year}/acs/{dataset}"
zcta_variables = "B01003_001E,B19013_001E,B25077_001E,B25064_001E,NAME"

params_zcta = {
    "get": zcta_variables,
    "for": "zip code tabulation area:*",
}
if CENSUS_API_KEY:
    params_zcta["key"] = CENSUS_API_KEY

print("Fetching ZCTA-level data for entire US (this may take a moment)...")
r_zcta = requests.get(zcta_base_url, params=params_zcta, timeout=120)
print(f"Status: {r_zcta.status_code}")
r_zcta.raise_for_status()

data_zcta = r_zcta.json()
zcta_cols = data_zcta[0]
zcta_rows = data_zcta[1:]

df_zcta = pd.DataFrame(zcta_rows, columns=zcta_cols)
df_zcta.rename(columns=rename_map, inplace=True)
for col in rename_map.values():
    df_zcta[col] = pd.to_numeric(df_zcta[col], errors="coerce")

# Rename the ZCTA column for clarity
zcta_col_name = [c for c in df_zcta.columns if "zip code" in c.lower() or "zcta" in c.lower()]
if zcta_col_name:
    df_zcta.rename(columns={zcta_col_name[0]: "zcta"}, inplace=True)
else:
    # Fallback: last column is usually the geography
    df_zcta.rename(columns={df_zcta.columns[-1]: "zcta"}, inplace=True)

print(f"\n--- Census ACS {year} -- ZCTA Level ({len(df_zcta)} ZCTAs) ---")
print(f"SUCCESS! Retrieved {len(df_zcta)} ZIP Code Tabulation Areas.")
print(f"\nZCTA-level summary:")
print(df_zcta[["population", "median_income", "median_home_value", "median_rent"]].describe().round(0))
df_zcta.head(10)

In [ ]:
# ---- Map 8: ZCTA-Level Median Income -- Scatter map (bubble plot on lat/lon) ----
# Since ZCTAs don't have a built-in GeoJSON in Plotly, we use a scatter geo approach.
# We'll map each ZCTA to its approximate centroid using the uszipcode library or a simple lookup.
# Alternative: use Plotly density mapbox for smooth rendering of ~33k points.

# First, try to get ZCTA centroids from Census Gazetteer files
print("Downloading ZCTA centroid data from Census Gazetteer...")
gazetteer_url = "https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2020_Gazetteer/2020_Gaz_zcta_national.txt"
try:
    df_gaz = pd.read_csv(gazetteer_url, sep="\t", dtype={"GEOID": str})
    df_gaz.columns = df_gaz.columns.str.strip()
    df_gaz.rename(columns={"GEOID": "zcta", "INTPTLAT": "lat", "INTPTLONG": "lon"}, inplace=True)
    df_gaz["zcta"] = df_gaz["zcta"].str.strip()
    print(f"Loaded {len(df_gaz)} ZCTA centroids from Census Gazetteer")
except Exception as e:
    print(f"Gazetteer download failed: {e}")
    print("Falling back to data without coordinates...")
    df_gaz = None

if df_gaz is not None:
    # Merge ZCTA demographics with centroid coordinates
    df_zcta_geo = df_zcta.merge(df_gaz[["zcta", "lat", "lon"]], on="zcta", how="inner")
    print(f"Matched {len(df_zcta_geo)} ZCTAs with coordinates (out of {len(df_zcta)})")
    
    # Filter valid data
    df_zcta_geo = df_zcta_geo[
        (df_zcta_geo["median_income"] > 0) & 
        (df_zcta_geo["population"] > 0) &
        (df_zcta_geo["lat"].between(24, 50)) &   # Continental US lat bounds
        (df_zcta_geo["lon"].between(-125, -66))   # Continental US lon bounds
    ].copy()
    print(f"After filtering: {len(df_zcta_geo)} ZCTAs in continental US with valid data")

In [ ]:
# ---- Map 9: ZCTA Median Income -- Scatter Geo (Continental US) ----
if df_gaz is not None and len(df_zcta_geo) > 0:
    fig_zcta_income = px.scatter_geo(
        df_zcta_geo,
        lat="lat",
        lon="lon",
        color="median_income",
        size="population",
        size_max=8,
        hover_name="NAME",
        hover_data={
            "zcta": True,
            "median_income": ":$,.0f",
            "population": ":,.0f",
            "median_home_value": ":$,.0f",
            "median_rent": ":$,.0f",
            "lat": False,
            "lon": False,
        },
        color_continuous_scale="Viridis",
        scope="usa",
        title=f"Census ACS {year} -- Median Income by ZIP Code (ZCTA) -- {len(df_zcta_geo):,} ZCTAs",
        labels={"median_income": "Median Income ($)", "zcta": "ZCTA"},
        opacity=0.6,
    )
    fig_zcta_income.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
        margin=dict(l=0, r=0, t=50, b=0),
        width=1100,
        height=650,
    )
    fig_zcta_income.show()
    print(f"Plotted {len(df_zcta_geo):,} ZCTAs (bubble size = population)")
else:
    print("ZCTA geo data not available, skipping scatter map")

In [ ]:
# ---- Map 10: ZCTA Median Home Value -- Scatter Geo ----
if df_gaz is not None and len(df_zcta_geo) > 0:
    df_zcta_hv = df_zcta_geo[df_zcta_geo["median_home_value"] > 0].copy()
    
    fig_zcta_hv = px.scatter_geo(
        df_zcta_hv,
        lat="lat",
        lon="lon",
        color="median_home_value",
        size="population",
        size_max=8,
        hover_name="NAME",
        hover_data={
            "zcta": True,
            "median_home_value": ":$,.0f",
            "median_income": ":$,.0f",
            "population": ":,.0f",
            "lat": False,
            "lon": False,
        },
        color_continuous_scale="Blues",
        range_color=[0, df_zcta_hv["median_home_value"].quantile(0.95)],
        scope="usa",
        title=f"Census ACS {year} -- Median Home Value by ZIP Code (ZCTA) -- {len(df_zcta_hv):,} ZCTAs",
        labels={"median_home_value": "Home Value ($)", "zcta": "ZCTA"},
        opacity=0.6,
    )
    fig_zcta_hv.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
        margin=dict(l=0, r=0, t=50, b=0),
        width=1100,
        height=650,
    )
    fig_zcta_hv.show()

In [ ]:
# ---- ZCTA Top/Bottom Analysis ----
print("=" * 80)
print(f"ZCTA (ZIP CODE) DEEP DIVE -- Census ACS {year}")
print("=" * 80)

df_zcta_valid = df_zcta[(df_zcta["median_income"] > 0) & (df_zcta["population"] > 0)].copy()
print(f"\nTotal ZCTAs: {len(df_zcta)} | Valid (income > 0, pop > 0): {len(df_zcta_valid)}")

print(f"\n--- Top 20 ZIP Codes by Median Income ---")
top20_zcta = df_zcta_valid.nlargest(20, "median_income")[["NAME", "zcta", "median_income", "population", "median_home_value"]].copy()
top20_zcta["median_income"] = top20_zcta["median_income"].apply(lambda x: f"${x:,.0f}")
top20_zcta["population"] = top20_zcta["population"].apply(lambda x: f"{x:,.0f}")
top20_zcta["median_home_value"] = top20_zcta["median_home_value"].apply(lambda x: f"${x:,.0f}" if pd.notna(x) and x > 0 else "N/A")
print(top20_zcta.to_string(index=False))

print(f"\n--- Bottom 20 ZIP Codes by Median Income (pop > 1,000) ---")
df_zcta_pop = df_zcta_valid[df_zcta_valid["population"] > 1000]
bot20_zcta = df_zcta_pop.nsmallest(20, "median_income")[["NAME", "zcta", "median_income", "population", "median_home_value"]].copy()
bot20_zcta["median_income"] = bot20_zcta["median_income"].apply(lambda x: f"${x:,.0f}")
bot20_zcta["population"] = bot20_zcta["population"].apply(lambda x: f"{x:,.0f}")
bot20_zcta["median_home_value"] = bot20_zcta["median_home_value"].apply(lambda x: f"${x:,.0f}" if pd.notna(x) and x > 0 else "N/A")
print(bot20_zcta.to_string(index=False))

print(f"\n--- Most Expensive ZIP Codes (by Home Value) ---")
df_zcta_hvf = df_zcta_valid[df_zcta_valid["median_home_value"] > 0]
top20_hv_zcta = df_zcta_hvf.nlargest(20, "median_home_value")[["NAME", "zcta", "median_home_value", "median_income", "population"]].copy()
top20_hv_zcta["median_home_value"] = top20_hv_zcta["median_home_value"].apply(lambda x: f"${x:,.0f}")
top20_hv_zcta["median_income"] = top20_hv_zcta["median_income"].apply(lambda x: f"${x:,.0f}")
top20_hv_zcta["population"] = top20_hv_zcta["population"].apply(lambda x: f"{x:,.0f}")
print(top20_hv_zcta.to_string(index=False))

---
## 8. FRED API -- Federal Reserve Economic Data

Pull key macro-economic time series from FRED, then map **state-level unemployment rates** down to the ZIP code level using the ZCTA-to-state mapping.

**Series:** CPI, 10Y Treasury, National Unemployment, Housing Starts, plus per-state unemployment rates.

In [ ]:
# ---- FRED: Pull macro time series ----
from fredapi import Fred

FRED_API_KEY = os.environ.get("FRED_API_KEY", "")
print("FRED API key:", "SET" if FRED_API_KEY else "NOT SET")

fred = Fred(api_key=FRED_API_KEY)

# National macro series
series_map = {
    "CPIAUCSL": "CPI (All Urban Consumers)",
    "DGS10": "10-Year Treasury Rate",
    "UNRATE": "National Unemployment Rate",
    "HOUST": "Housing Starts (thousands)",
}

fred_data = {}
print("\nFetching FRED national macro series...")
for sid, label in series_map.items():
    try:
        s = fred.get_series(sid, observation_start="2020-01-01")
        fred_data[sid] = s
        latest = s.dropna().iloc[-1]
        latest_date = s.dropna().index[-1].strftime("%Y-%m-%d")
        print(f"  [OK] {label:35s} | Latest: {latest:>10.2f} ({latest_date}) | {len(s)} obs")
    except Exception as e:
        print(f"  [ERR] {label:35s} | {e}")

print(f"\nSUCCESS! Retrieved {len(fred_data)} national series from FRED.")


In [ ]:
# ---- FRED: Plot macro time series ----
from plotly.subplots import make_subplots

fig_fred = make_subplots(
    rows=2, cols=2,
    subplot_titles=list(series_map.values()),
    vertical_spacing=0.12, horizontal_spacing=0.08,
)

colors = ["#1f77b4", "#ff7f0e", "#d62728", "#2ca02c"]
positions = [(1,1), (1,2), (2,1), (2,2)]

for idx, (sid, label) in enumerate(series_map.items()):
    if sid in fred_data:
        s = fred_data[sid].dropna()
        r, c = positions[idx]
        fig_fred.add_trace(
            go.Scatter(x=s.index, y=s.values, name=label,
                       line=dict(color=colors[idx], width=2)),
            row=r, col=c
        )

fig_fred.update_layout(
    title="FRED Macro Economic Indicators (2020-present)",
    height=600, width=1000,
    showlegend=False,
    margin=dict(t=80),
)
fig_fred.show()


In [ ]:
# ---- FRED: Pull state-level unemployment rates ----
# FRED series: {STATE_ABBR}UR (e.g. NYUR, CAUR)
state_abbr_to_fips = {v: k for k, v in fips_to_abbr.items()}

state_ur_series = {}
print("Fetching state-level unemployment rates from FRED...")
errors_fred = []
for abbr in sorted(fips_to_abbr.values()):
    if abbr in ("PR", "DC"):
        continue
    sid = f"{abbr}UR"
    try:
        s = fred.get_series(sid, observation_start="2023-01-01")
        if len(s.dropna()) > 0:
            state_ur_series[abbr] = s.dropna().iloc[-1]
    except Exception as e:
        errors_fred.append(f"{sid}: {e}")

print(f"Retrieved unemployment rates for {len(state_ur_series)} states")
if errors_fred:
    print(f"Errors: {len(errors_fred)} (first 3: {errors_fred[:3]})")

df_state_ur = pd.DataFrame([
    {"state_abbr": abbr, "unemployment_rate": rate}
    for abbr, rate in state_ur_series.items()
])
df_state_ur.sort_values("unemployment_rate", ascending=False, inplace=True)
print(f"\nTop 10 states by unemployment:")
print(df_state_ur.head(10).to_string(index=False))
print(f"\nBottom 10 states by unemployment:")
print(df_state_ur.tail(10).to_string(index=False))


In [ ]:
# ---- Map 11: State Unemployment from FRED ----
fig_ur_state = px.choropleth(
    df_state_ur,
    locations="state_abbr",
    locationmode="USA-states",
    color="unemployment_rate",
    hover_data={"unemployment_rate": ":.1f", "state_abbr": True},
    color_continuous_scale="RdYlGn_r",
    scope="usa",
    title="FRED -- Latest State Unemployment Rates (%)",
    labels={"unemployment_rate": "Unemp Rate (%)", "state_abbr": "State"},
)
fig_ur_state.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=950, height=550,
)
fig_ur_state.show()


In [ ]:
# ---- Map 12: FRED Unemployment mapped to ZIP code level ----
# Map state unemployment to each ZCTA using Gazetteer state column
# The Gazetteer df_gaz was loaded in section 7 (ZCTA centroid download)

try:
    gaz_cols = df_gaz.columns.tolist()
    # Find state abbreviation column in Gazetteer
    state_col = None
    for col in gaz_cols:
        if col.upper() in ("USPS", "STATE"):
            state_col = col
            break
    
    if state_col:
        df_zcta_state = df_gaz[["zcta", state_col, "lat", "lon"]].copy()
        df_zcta_state.rename(columns={state_col: "state_abbr"}, inplace=True)
        df_zcta_state["state_abbr"] = df_zcta_state["state_abbr"].str.strip()
    else:
        print("Gazetteer columns:", gaz_cols)
        raise ValueError("No state column found in Gazetteer")
    
    # Merge unemployment rates onto ZCTAs
    df_zcta_ur = df_zcta_state.merge(df_state_ur, on="state_abbr", how="inner")
    df_zcta_ur = df_zcta_ur.merge(
        df_zcta[["zcta", "population", "median_income"]],
        on="zcta", how="inner"
    )
    # Filter to continental US
    df_zcta_ur = df_zcta_ur[
        (df_zcta_ur["lat"].between(24, 50)) &
        (df_zcta_ur["lon"].between(-125, -66)) &
        (df_zcta_ur["population"] > 0)
    ].copy()
    
    print(f"Mapped FRED unemployment to {len(df_zcta_ur):,} ZCTAs")
    
    fig_zcta_ur = px.scatter_geo(
        df_zcta_ur,
        lat="lat", lon="lon",
        color="unemployment_rate",
        size="population",
        size_max=7,
        hover_data={
            "zcta": True,
            "state_abbr": True,
            "unemployment_rate": ":.1f",
            "population": ":,.0f",
            "median_income": ":$,.0f",
            "lat": False, "lon": False,
        },
        color_continuous_scale="RdYlGn_r",
        scope="usa",
        title=f"FRED State Unemployment Mapped to ZIP Code Level -- {len(df_zcta_ur):,} ZCTAs",
        labels={"unemployment_rate": "Unemp Rate (%)", "zcta": "ZCTA"},
        opacity=0.5,
    )
    fig_zcta_ur.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
        margin=dict(l=0, r=0, t=50, b=0),
        width=1100, height=650,
    )
    fig_zcta_ur.show()
except NameError:
    print("df_gaz not available -- make sure to run ZCTA section (Section 7) first")
except Exception as e:
    print(f"Error mapping FRED to ZCTA: {e}")


---
## 9. OpenCelliD -- Cell Tower Data at Lat/Lon Level

Pull cell tower locations from OpenCelliD for **multiple US metro areas**, then map every tower at its exact lat/lon with radio technology breakdown (LTE, GSM, UMTS, CDMA, 5G NR).

**API:** `https://opencellid.org/cell/getInArea` (bounding box query, 1000 req/day free tier)

In [ ]:
# ---- OpenCelliD: Pull cell towers for major US metro areas ----
import time as _time

OPENCELLID_API_KEY = os.environ.get("OPENCELLID_API_KEY", "")
print("OpenCelliD API key:", "SET" if OPENCELLID_API_KEY else "NOT SET")

all_towers = []
df_towers = pd.DataFrame()

if not OPENCELLID_API_KEY:
    print("Set OPENCELLID_API_KEY in .env to run this section")
else:
    # OpenCelliD limits bbox to ~4 sq km (~0.02 x 0.02 degrees)
    # Use small downtown bboxes centered on major metros
    # Each bbox is ~0.01 lat x 0.01 lon (approx 1km x 1km)
    metro_bboxes = [
        ("NYC - Midtown",        40.748, -73.990, 40.758, -73.980),
        ("NYC - Downtown",       40.708, -74.015, 40.718, -74.005),
        ("LA - Downtown",        34.047, -118.260, 34.057, -118.250),
        ("LA - Hollywood",       34.098, -118.340, 34.108, -118.330),
        ("Chicago - Loop",       41.878, -87.635, 41.888, -87.625),
        ("Chicago - North",      41.940, -87.655, 41.950, -87.645),
        ("Houston - Downtown",   29.755, -95.370, 29.765, -95.360),
        ("Phoenix - Downtown",   33.445, -112.080, 33.455, -112.070),
        ("Philly - Center",      39.950, -75.170, 39.960, -75.160),
        ("Dallas - Downtown",    32.775, -96.802, 32.785, -96.792),
        ("SF - Financial",       37.790, -122.405, 37.800, -122.395),
        ("DC - Capitol",         38.888, -77.015, 38.898, -77.005),
        ("Miami - Downtown",     25.770, -80.200, 25.780, -80.190),
        ("Atlanta - Downtown",   33.748, -84.395, 33.758, -84.385),
        ("Boston - Downtown",    42.355, -71.065, 42.365, -71.055),
    ]
    
    url_oci = "https://opencellid.org/cell/getInArea"
    
    print(f"\nFetching cell towers for {len(metro_bboxes)} metro areas...")
    print("(Using ~1km x 1km bounding boxes -- OpenCelliD limit is ~4 sq km)\n")
    
    for name, min_lat, min_lon, max_lat, max_lon in metro_bboxes:
        bbox_str = f"{min_lat},{min_lon},{max_lat},{max_lon}"
        params_oci = {"key": OPENCELLID_API_KEY, "BBOX": bbox_str, "format": "json", "limit": 1000}
        try:
            r_oci = requests.get(url_oci, params=params_oci, timeout=30)
            if r_oci.status_code == 200:
                data_oci = r_oci.json()
                if "error" in data_oci:
                    print(f"  [ERR] {name:25s} | {data_oci['error']}")
                    continue
                cells_list = data_oci.get("cells", data_oci if isinstance(data_oci, list) else [])
                for cell in cells_list:
                    cell["metro"] = name
                all_towers.extend(cells_list)
                radios = pd.Series([c.get("radio", "") for c in cells_list]).value_counts()
                radio_str = ", ".join([f"{k}:{v}" for k, v in radios.items()])
                print(f"  [OK] {name:25s} | {len(cells_list):>4} towers | {radio_str}")
            else:
                print(f"  [ERR] {name:25s} | HTTP {r_oci.status_code}: {r_oci.text[:100]}")
        except Exception as e:
            print(f"  [ERR] {name:25s} | {e}")
        _time.sleep(1)  # Rate limit courtesy
    
    print(f"\nTotal towers retrieved: {len(all_towers)}")
    
    if all_towers:
        df_towers = pd.DataFrame(all_towers)
        if "lng" in df_towers.columns and "lon" not in df_towers.columns:
            df_towers.rename(columns={"lng": "lon"}, inplace=True)
        print(f"\nRadio technology distribution (all metros):")
        print(df_towers["radio"].value_counts().to_string())
        print(f"\nTowers per metro area:")
        print(df_towers["metro"].value_counts().to_string())


In [ ]:
# ---- Map 13: OpenCelliD Cell Towers -- All Metros at Lat/Lon ----
if OPENCELLID_API_KEY and len(all_towers) > 0:
    radio_colors = {
        "LTE": "#1f77b4",
        "GSM": "#ff7f0e",
        "UMTS": "#2ca02c",
        "CDMA": "#d62728",
        "NR": "#9467bd",
    }
    
    fig_towers = go.Figure()
    
    for radio_type in df_towers["radio"].unique():
        df_radio = df_towers[df_towers["radio"] == radio_type]
        fig_towers.add_trace(go.Scattergeo(
            lon=df_radio["lon"].tolist(),
            lat=df_radio["lat"].tolist(),
            text=(df_radio['metro'] + ' | ' + df_radio['radio']).tolist(),
            mode="markers",
            marker=dict(
                size=5,
                color=radio_colors.get(radio_type, "#999999"),
                opacity=0.7,
                line=dict(width=0.5, color="white"),
            ),
            name=f"{radio_type} ({len(df_radio)})",
            hoverinfo="text",
        ))
    
    fig_towers.update_layout(
        title='OpenCelliD Cell Towers -- ' + str(len(df_towers)) + ' Towers across ' + str(df_towers['metro'].nunique()) + ' US Metros',
        geo=dict(
            scope="usa",
            bgcolor="rgba(0,0,0,0)",
            lakecolor="rgb(200,220,240)",
            showlakes=True,
            landcolor="rgb(243,243,243)",
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        width=1100, height=650,
        legend=dict(title="Radio Tech", yanchor="top", y=0.99, xanchor="left", x=0.01),
    )
    fig_towers.show()
    print(f"Mapped {len(df_towers):,} cell towers colored by radio technology")
else:
    print("No tower data available")


In [ ]:
# ---- Map 14: OpenCelliD Zoomed Metro Views ----
if OPENCELLID_API_KEY and len(all_towers) > 0:
    from plotly.subplots import make_subplots
    top_metros = df_towers["metro"].value_counts().head(4).index.tolist()
    
    fig_metro_zoom = make_subplots(
        rows=2, cols=2,
        subplot_titles=top_metros,
        specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}],
               [{"type": "scattergeo"}, {"type": "scattergeo"}]],
        vertical_spacing=0.05, horizontal_spacing=0.02,
    )
    
    for idx, metro in enumerate(top_metros):
        row, col = (idx // 2) + 1, (idx % 2) + 1
        df_m = df_towers[df_towers["metro"] == metro]
        
        for radio_type in df_m["radio"].unique():
            df_r = df_m[df_m["radio"] == radio_type]
            fig_metro_zoom.add_trace(
                go.Scattergeo(
                    lon=df_r["lon"].tolist(),
                    lat=df_r["lat"].tolist(),
                    mode="markers",
                    marker=dict(size=4, color=radio_colors.get(radio_type, "#999"), opacity=0.8),
                    name=radio_type,
                    showlegend=(idx == 0),
                ),
                row=row, col=col
            )
        
        center_lat = df_m["lat"].mean()
        center_lon = df_m["lon"].mean()
        geo_key = f"geo{idx+1}" if idx > 0 else "geo"
        fig_metro_zoom.update_layout(**{
            geo_key: dict(
                scope="usa",
                center=dict(lat=center_lat, lon=center_lon),
                projection_scale=25,
                bgcolor="rgba(0,0,0,0)",
                landcolor="rgb(243,243,243)",
            )
        })
    
    fig_metro_zoom.update_layout(
        title="OpenCelliD -- Top 4 Metro Areas (Zoomed)",
        height=800, width=1100,
        margin=dict(l=0, r=0, t=80, b=0),
    )
    fig_metro_zoom.show()
    
    print("\n--- Tower Count by Metro & Radio Technology ---")
    pivot = df_towers.groupby(["metro", "radio"]).size().unstack(fill_value=0)
    pivot["TOTAL"] = pivot.sum(axis=1)
    pivot.sort_values("TOTAL", ascending=False, inplace=True)
    print(pivot.to_string())


---
## 6. Visualize -- Geocoded Locations on US Map

Plot the geocoded test locations as scatter points overlaid on the median income choropleth.

In [ ]:
# ---- Combined map: income choropleth + geocoded locations ----
df_geo_ok = df_geocoded[df_geocoded["status"] == "OK"].copy()

fig_combined = go.Figure()

# Base: income choropleth
fig_combined.add_trace(go.Choropleth(
    locations=df_map["state_abbr"],
    z=df_map["median_income"],
    locationmode="USA-states",
    colorscale="Viridis",
    colorbar=dict(title="Median<br>Income ($)", x=1.0),
    text=df_map["NAME"],
    hovertemplate="<b>%{text}</b><br>Median Income: $%{z:,.0f}<extra></extra>",
))

# Overlay: geocoded points
if len(df_geo_ok) > 0:
    hover_texts = []
    for _, r in df_geo_ok.iterrows():
        hover_texts.append(f"{r['location']}<br>GEOID: {r.get('geoid','')}<br>Tract: {r.get('tract','')}")
    
    fig_combined.add_trace(go.Scattergeo(
        lon=df_geo_ok["lon"].tolist(),
        lat=df_geo_ok["lat"].tolist(),
        text=hover_texts,
        mode="markers",
        marker=dict(size=14, color="red", line=dict(width=2, color="white"), symbol="star"),
        name="Geocoded Locations",
        hovertemplate="<b>%{text}</b><extra></extra>",
        showlegend=True,
    ))
    print(f"Plotted {len(df_geo_ok)} geocoded locations on map")
else:
    print("No successfully geocoded locations to plot")

fig_combined.update_layout(
    title=f"Census ACS {year} Median Income + Geocoded Test Locations",
    geo=dict(
        scope="usa",
        bgcolor="rgba(0,0,0,0)",
        lakecolor="rgb(200,220,240)",
        showlakes=True,
    ),
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000,
    height=600,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig_combined.show()

---
## 7. Visualize -- Median Rent by State

In [ ]:
# ---- Map 4: Median Gross Rent by State ----
fig_rent = px.choropleth(
    df_map,
    locations="state_abbr",
    locationmode="USA-states",
    color="median_rent",
    hover_name="NAME",
    hover_data={
        "median_rent": ":$,.0f",
        "median_income": ":$,.0f",
        "population": ":,.0f",
        "state_abbr": False,
    },
    color_continuous_scale="Plasma",
    scope="usa",
    title=f"Census ACS {year} -- Median Gross Rent by State",
    labels={"median_rent": "Median Rent ($)"},
)
fig_rent.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)", lakecolor="rgb(200,220,240)"),
    margin=dict(l=0, r=0, t=50, b=0),
    width=900,
    height=550,
)
fig_rent.show()

---
## 8. Summary Statistics

In [ ]:
# ---- Final Summary ----
print("=" * 80)
print("FULL API TEST SUMMARY")
print("=" * 80)

print(f"\n  Source           | Level      | Records")
print(f"  -----------------+------------+-------------")
print(f"  Census ACS       | State      | {len(df_states):>9,}")
print(f"  Census ACS       | County     | {len(df_counties):>9,}")
print(f"  Census ACS       | Tract (NY) | {len(df_tracts_ny):>9,}")
print(f"  Census ACS       | ZCTA (ZIP) | {len(df_zcta):>9,}")
print(f"  Census Geocoder  | Point      | {ok_count}/{len(df_geocoded)} matched")
print(f"  FRED             | National   | {len(fred_data)} series")
print(f"  FRED             | State UR   | {len(state_ur_series)} states")

tower_count = len(df_towers) if len(df_towers) > 0 else 0
tower_metros = df_towers["metro"].nunique() if tower_count > 0 else 0
print(f"  OpenCelliD       | Lat/Lon    | {tower_count:>9,}")

print(f"\n  Maps generated: 14 total")
print(f"    Census:     3 state + 3 county + 2 ZCTA + 1 combined + 1 rent = 10")
print(f"    FRED:       1 state unemployment + 1 ZCTA unemployment = 2")
print(f"    OpenCelliD: 1 national overview + 1 metro zoom = 2")

print(f"\n  Geographic hierarchy:")
print(f"    State ({len(df_states)}) -> County ({len(df_counties):,}) -> ZIP/ZCTA ({len(df_zcta):,}) -> Tract ({len(df_tracts_ny):,} NY)")
if tower_count > 0:
    print(f"    Cell towers: {tower_count:,} across {tower_metros} metros (exact lat/lon)")

print("\n" + "=" * 80)
print("ALL API TESTS PASSED -- Census + FRED + OpenCelliD working!")
print("=" * 80)
